In [202]:
import bs4
import time
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
import re
import requests as rq


In [203]:
link = 'https://fastestlaps.com/tracks/nordschleife'
addr = "https://fastestlaps.com"

In [215]:
res = rq.get(link)                                      # request로 해당링크로 접속
my_html = res.text                        
my_soup = bs4.BeautifulSoup(my_html, 'html.parser')    

vehicle = my_soup.select("#laptimes > table > tbody > tr > td ")

In [216]:
dic ={"vehicle" : [] , "laptime" : [] , "url" : []}     # URL , 차종 , laptime 크롤링
for i in range(len(vehicle)):
    if i%5==1:
        dic["vehicle"].append(vehicle[i].text)
        if vehicle[i].select("a") ==[]:
            dic["url"].append((vehicle[i].select("a")))
        else : 
            f = vehicle[i].find("a")
            dic["url"].append(f["href"])
            
    if i%5==3:
        dic["laptime"].append(vehicle[i].text)

In [220]:
url_list = dic["url"]
spec_dic = {"CurbWeight" : [] , "TopSpeed" : [],"Displacement" : [], "Power" : [] , "Torque" : [],"Zero":[]  }
for url in url_list:
    if url ==[] :                                          # URL이 없을 경우 결측치로 입력
        spec_dic["CurbWeight"].append('')
        spec_dic["TopSpeed"].append('')
        spec_dic["Displacement"].append('')
        spec_dic["Power"].append('')
        spec_dic["Torque"].append('')
        spec_dic["Zero"].append('')
        continue
        
    res = rq.get(addr+url)                                  # 크롤링 한 URL로 접속
    my_html = res.text                                      # 사이트 내용을 가져온다.
    my_soup = bs4.BeautifulSoup(my_html, 'html.parser')     # BeautifulSoup 객체 반환. 'html.parser' 사용.
    spec = my_soup.select("tr > td ")                       # td태그 고르기
    Power, TopSpeed , Torque, CurbWeight, Displacement,Zero = "", "" ,"" ,"" ,"",""
    for i in range(len(spec)):                              # 각 변수에 해당하는 값 입력하기
        if (spec[i].text == "Power"):
            Power = re.findall("\d+.\d+",spec[i+1].text)[0]
        if (spec[i].text == "Top speed"):
            TopSpeed = re.findall("\d+.\d+",spec[i+1].text)[0]
        if (spec[i].text == "Torque"):
            Torque = re.findall("\d+.\d+",spec[i+1].text)[0]
        if (spec[i].text == "Curb weight"):
            CurbWeight = re.findall("\d+",spec[i+1].text)[0]
        if (spec[i].text == "Displacement"):
            Displacement = re.findall("\d+.\d+",spec[i+1].text)[0]
        if (spec[i].text == "0 - 100 kph"):
            Zero = re.findall("\d+.\d+",spec[i+1].text)[0]
    spec_dic["CurbWeight"].append(CurbWeight)
    spec_dic["TopSpeed"].append(TopSpeed)
    spec_dic["Displacement"].append(Displacement)
    spec_dic["Power"].append(Power)
    spec_dic["Torque"].append(Torque)
    spec_dic["Zero"].append(Zero)

In [221]:
df = pd.DataFrame(spec_dic)                                # 크롤링한 데이터를 DataFrame으로 만들어준다

In [222]:
df["vehicle"] = dic["vehicle"]                             # 앞에서 크롤링한 Laptime, 차종을 합쳐준다.
df["laptime"] = dic["laptime"]

In [223]:
df.to_excel("vehicle_zero.xlsx")                            # 크롤링한 데이터를 Excel파일로만들어준다.

In [217]:
# data = pd.read_excel("vehicle.xlsx")
dic["url"]

[[],
 '/models/mercedes-benz-amg-gt-r-black-series',
 '/models/lamborghini-aventador-svj',
 '/models/radical-sr8lm',
 '/models/lamborghini-huracn-performante',
 '/models/radical-sr8',
 '/models/porsche-911-gt3-rs-991-facelift',
 '/models/porsche-918-spyder',
 '/models/subaru-wrx-sti',
 '/models/lamborghini-aventador-lp750-4-sv',
 '/models/ferrari-488-pista',
 '/models/dodge-viper-acr-mk-v',
 '/models/mercedes-benz-amg-gt-r-pro',
 '/models/nio-ep9',
 '/models/mercedes-benz-amg-gt-r-pro',
 '/models/mclaren-720s',
 '/models/nissan-gt-r-nismo',
 '/models/mclaren-600lt',
 '/models/mercedes-benz-amg-gt-r',
 '/models/gumpert-apollo-sport',
 '/models/dodge-viper-srt-10-acr',
 '/models/porsche-911-gt3-991-facelift',
 '/models/porsche-918-spyder',
 '/models/chevrolet-corvette-z06-c7',
 '/models/mclaren-720s',
 '/models/lexus-lfa-nurburgring-package',
 '/models/donkervoort-d8-rs',
 '/models/chevrolet-camaro-g6-zl1-1le',
 '/models/porsche-911-turbo-s-992',
 '/models/porsche-911-turbo-s-991-facelif

In [201]:
data.isnull().sum()

Unnamed: 0      0
CurbWeight      2
TopSpeed        6
Displacement    4
Power           0
Torque          7
vehicle         0
laptime         0
dtype: int64